In [63]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from load_data import merged_df
from window_generator_cnn_predict import WindowGenerator

In [64]:
os.chdir("/Users/jacksonwalters/Documents/GitHub/enefit-kaggle/predict-energy-behavior-of-prosumers/")

In [65]:
#load the model using the legacy .h5 save method
cnn_model = tf.keras.models.load_model('../models/cnn_model.keras')

In [66]:
cnn_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_5 (Bat  (None, 12, 24)            96        
 chNormalization)                                                
                                                                 
 conv1d_5 (Conv1D)           (None, 1, 32)             9248      
                                                                 
 dense_10 (Dense)            (None, 1, 32)             1056      
                                                                 
 dense_11 (Dense)            (None, 1, 1)              33        
                                                                 
 reshape_4 (Reshape)         (None,)                   0         
                                                                 
Total params: 10433 (40.75 KB)
Trainable params: 10385 (40.57 KB)
Non-trainable params: 48 (192.00 Byte)
_______________

In [4]:
df = merged_df()

loading train data...
loading gas_prices...
loading electricity_prices...
loading forecast_weather...
merging train and gas_prices...
merging electricity_prices...
merging forecast_weather...


In [67]:
#create a prediction df with all target values set to zero
predict_df = df[:100_000].copy()

In [68]:
#set the target to zero. this means we don't have any prior knowledge of the targets
predict_df['target'] = 0.

In [69]:
#multi-step dense network
CONV_WIDTH = 12
conv_window = WindowGenerator(
    predict_df = predict_df,
    input_width=CONV_WIDTH,
    label_width=1,
    shift=1,
    label_columns=None)

In [70]:
print('Input shape:', conv_window.example[0].shape)
print('Output shape:', cnn_model(conv_window.example[0]).shape)

Input shape: (32, 12, 24)
Output shape: (32,)


In [71]:
#note that the train, test, validation data all have a second "label" representing the target value
cnn_model.predict(conv_window.example[0])

1/1 [==============================] - 0s 124ms/step


array([4.4236937, 3.5083323, 3.614612 , 3.4118683, 4.1260114, 4.6097374,
       4.059664 , 4.669627 , 3.836762 , 4.6434197, 3.9667733, 3.7722063,
       3.2709942, 3.0424068, 3.924722 , 3.4995837, 3.7406573, 4.0532827,
       4.3338447, 4.036341 , 2.7434535, 3.6825666, 3.6922872, 3.3697805,
       3.8178082, 2.5590231, 3.7498977, 3.2020311, 3.446014 , 3.3464618,
       4.630179 , 3.1024594], dtype=float32)

In [72]:
pred=cnn_model.predict(conv_window.predict)

3125/3125 [==============================] - 12s 4ms/step
